In [1]:
import os
from sklearn.feature_extraction.text import CountVectorizer
import lda
import numpy as np
import pickle
import seaborn as sns

from matplotlib import pyplot as plt

In [2]:
root = "/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/"

In [3]:
filename = root + "done_processing.txt"
rating_file = root + "rating.txt"
source = root + "source.txt"

source_name = []
corpus = []
page_id = []
doc_count = 0
rating = []
blank = []

with open(filename, 'r') as f:
    line = f.readline()
    while(line):
        if(line[0]!="*"):
            corpus.append(line)
        elif(lens(line)==0):
            print("wtf?")
            # doc_count = doc_count +1
        line = f.readline()
with open(rating_file, 'r') as f2:
    line = f2.readline()
    while(line):
        if(line[0]=="*"):
            page_id.append(doc_count)#-len(page_id))
        else:
            rating.append(np.int64(line))
            doc_count = doc_count +1
        line = f2.readline()
with open(source, 'r') as f3:
    line = f3.readline()
    while(line):
        if(line[0]=="*"):
#             page_id.append(doc_count-len(page_id))
            a = 6
        else:
            source_name.append(line)
            doc_count = doc_count +1
        line = f3.readline()
#         print(line)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names()
# print(vectorizer.get_feature_names())
# print(X.toarray())

model_filename = root+'finalized_model.sav'
model = pickle.load(open(model_filename, 'rb'))

page_vector = []
topic_word = model.topic_word_
doc_topic = model.doc_topic_

In [4]:
# topic_file = root + "topic.txt"
# with open(topic_file, 'w') as f:
#     n_top_words = 10
#     for i, topic_dist in enumerate(topic_word):
#         topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#         f.write('Topic {}: {}'.format(i, ' '.join(topic_words)))
#         f.write("\n")

In [5]:
sum = 0
for j in range(len(corpus)):
    sum = sum + rating[j]
for j in range(len(corpus)):
    rating[j] = rating[j]/sum
rating_topic = np.zeros((20,))
for i,vector in enumerate(doc_topic):
    rating_topic = rating_topic + vector*rating[i]
    

In [6]:
print(rating_topic)

[0.08050593 0.03127196 0.04256692 0.04898087 0.02799734 0.03203135
 0.02845676 0.02982411 0.08157834 0.02520465 0.09123937 0.09855209
 0.03048145 0.05560702 0.05422477 0.03631895 0.09041135 0.03581807
 0.03282676 0.04610195]


In [7]:
np.argmax(rating_topic)

11

In [8]:
topic_file = root + "topic.txt"
with open(topic_file, 'w') as f:
    n_top_words = 10
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
        f.write('Topic {}: {}'.format(i, ' '.join(topic_words)))
        f.write("\n")
    f.write("Hottest topic is: Topic "+ str(np.argmax(rating_topic)))
    